In [1]:
import json
import datetime as dt

import voltoolbox
from voltoolbox.fit.option_quotes import OptionSnapshot
from voltoolbox import bs_implied_volatility
from voltoolbox.calendar import nyse_calendar
from voltoolbox import BusinessTimeMeasure


file = 'vol_SPX_20210520_2011.json'

with open(file, 'r') as f:
    quotes_dict = json.loads(f.read())

quotes = OptionSnapshot.from_json_dict(quotes_dict)


In [ ]:
import pytz
import pandas as pd
import numpy as np

import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = np.array([6.0, 4.0]) * 2


pricing_dt = quotes.time_stamp
spot = quotes.ref_spot

for opt_sl in quotes.slices:
    calls = opt_sl.call
    plt.plot(calls.strikes, calls.bids, marker='+', linestyle='none')

    puts = opt_sl.put
    plt.plot(puts.strikes, puts.bids, marker='+', linestyle='none')

plt.axvline(x=quotes.ref_spot, color= 'black')
plt.grid()

In [ ]:
import pytz
import forward_fit
from forward_fit import filter_quotes, act365_time

business_time = BusinessTimeMeasure(nyse_calendar(), 0.5, 252.0)


slice_index_w = widgets.IntSlider(min=0, max =len(quotes.slices)-1)
@widgets.interact(slice_index=slice_index_w)
def plot_callput_parity(slice_index):
    pricing_dt = quotes.time_stamp
    if pricing_dt.tzinfo is None:
        pricing_dt = pytz.UTC.localize(pricing_dt)    

    discount = 1.0
    opt_sl = filter_quotes(quotes.slices[slice_index], 
                           pricing_dt,
                           discount,
                           spot, 
                           yield_threshold=0.05)
    
    call = opt_sl.call
    call_ks = np.array(call.strikes)
    call_bids = np.array(call.bids)
    call_asks = np.array(call.asks)
    

    put = opt_sl.put
    put_ks = np.array(put.strikes)
    put_bids = np.array(put.bids)
    put_asks = np.array(put.asks)
 
    
    ks = np.array(call.strikes + put.strikes)
    call_ones = np.array([1.0] * len(call.strikes) + [0.0] * len(put.strikes))
    put_ones = np.array([0.0] * len(call.strikes) + [1.0] * len(put.strikes))
    
    
    def convex(z):
        a, b, c = (-1.5, 0.0, 1.5)

        res = (max(0, z - a)**3 - max(0, z - b)**3) / (b - a)
        res -= (max(0, z - b)**3 - max(0, z - c)**3) / (c - b)
        res -= (b - a)**2
            
        res += (max(0, c - z)**3 - max(0, b - z)**3) / (c - b)
        res -= (max(0, b - z)**3 - max(0, a - z)**3) / (b - a)
        res -= (c-b)**2
        return res / 12.0  
    
    
    t = business_time.distance(pricing_dt, opt_sl.expiry)    
    dev = 0.15 * np.sqrt(max(1.0 / 365.0, t))
        
    ref_k = spot
    basis = np.column_stack([call_ones, 
                             put_ones,
                             ks - ref_k, 
                             np.vectorize(convex)( (ks/ref_k -1.0) / dev )])
    target = np.concatenate((0.5 * (call_bids + call_asks) / discount + call_ks, 
                             0.5 * (put_bids + put_asks) / discount
                             ), axis=0)
    
    ws = np.exp(-0.5 * ( (ks /  ref_k - 1  ) / (0.25 * dev)) **2)
    b_ws = np.column_stack( [ws] * 4) 
    var = np.matmul((basis * b_ws).T, basis * b_ws)
    cov = (basis * b_ws).T.dot(target * ws)
    coeffs = np.linalg.solve(var, cov)
    
    estimated_fwd = coeffs[0] - coeffs[1]
    
    vol_put_bids = np.array([max(0.0, bs_implied_volatility(estimated_fwd, k, p, t, -1.0))
                             for k, p in zip(put_ks, put_bids / discount)])
    vol_put_asks = np.array([max(0.0, bs_implied_volatility(estimated_fwd, k, p, t, -1.0))
                             for k, p in zip(put_ks, put_asks / discount)])
    
    vol_call_bids = np.array([max(0.0, bs_implied_volatility(estimated_fwd, k, p, t, 1.0))
                              for k, p in zip(call_ks, call_bids / discount)])
    vol_call_asks = np.array([max(0.0, bs_implied_volatility(estimated_fwd, k, p, t, 1.0))
                              for k, p in zip(call_ks, call_asks / discount)])

    plt.plot(put_ks, vol_put_bids, marker='^', linestyle='none')
    plt.plot(put_ks, vol_put_asks, marker='v', linestyle='none')
    plt.plot(call_ks, vol_call_bids, marker='^', linestyle='none')
    plt.plot(call_ks, vol_call_asks, marker='v', linestyle='none')

 #    plt.ylim((-50, 500))
#     plt.xlim((2000, 5000))
    
    plt.axvline(x=estimated_fwd, color = 'black')
    plt.title(f'{opt_sl.expiry}')
    plt.grid()